<a href="https://colab.research.google.com/github/guillotine46/sarcasm-detection-semeval2022/blob/main/Sarcasm_Task_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "tensorflow==1.15"

In [ ]:
!pip install "tensorflow_hub==0.6.0"

In [ ]:
pip install simpletransformers

In [ ]:
# load packages
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
import tensorflow.keras
from tensorflow.keras.layers import Input, Lambda, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import string
import pandas as pd
import re
import spacy
import nltk

from nltk.corpus import stopwords
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from spacy.lang.en import English
spacy.load('en')
parser = English()
from tqdm import tqdm

In [ ]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_hub as hub

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [ ]:
nltk.download('stopwords')

In [ ]:
# Stop words and special characters
STOPLIST = set(stopwords.words('english') + list(ENGLISH_STOP_WORDS))
SYMBOLS = " ".join(string.punctuation).split(" ") + ["-", "...", "”", "”","''"]

In [ ]:
def tokenizeText(text):

    text = text.strip().replace("\n", " ").replace("\r", " ")
    text = text.lower()

    tokens = parser(text)

    # lemmatization
    lemmas = []
    for tok in tokens:
        lemmas.append(tok.lemma_.lower().strip() if tok.lemma_ != "-PRON-" else tok.lower_)
    tokens = lemmas

    # reomve stop words and special charaters
    tokens = [tok for tok in tokens if tok.lower() not in STOPLIST]
    #tokens = [tok for tok in tokens if tok not in SYMBOLS]

    tokens = [tok for tok in tokens if len(tok) >= 3]

    # remove remaining tokens that are not alphabetic
    tokens = [tok for tok in tokens if tok.isalpha()]

    tokens = list(set(tokens))

    return ' '.join(tokens[:])

In [ ]:
def encode(le_enc, labels):
    enc = le_enc.transform(labels)
    return tensorflow.keras.utils.to_categorical(enc)

def decode(le_enc, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le_enc.inverse_transform(dec)

In [ ]:
df_train = pd.read_csv('Eng_train.csv',usecols=['tweet','sarcastic'],encoding='utf8')

In [ ]:
df_train.shape

(3468, 2)

In [ ]:
df_train['sarcastic'].unique()

array([1, 0])

In [ ]:
df_train = df_train.dropna()
df_train['tweet'] = df_train['tweet'].apply(lambda x:tokenizeText(x))

In [ ]:
# ELMo Embedding
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
x = df_train['tweet'].tolist()
y = df_train['sarcastic'].tolist()

le_enc = preprocessing.LabelEncoder()
le_enc.fit(y)

y_en = encode(le_enc, y)

In [ ]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(np.asarray(x), np.asarray(y_en), test_size=0.2, random_state=42)

In [ ]:
pip install h5py==2.10.0

In [ ]:
# Build Model
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=10, batch_size=16)
    model.save_weights('./response-elmo-model.h5')

with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')
    predicts = model.predict(x_test, batch_size=16)
    #predictions = model.predict(df)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Train on 2773 samples
Epoch 1/10
2773/2773 [==============================] - 394s 142ms/sample - loss: 0.5908 - acc: 0.7486
Epoch 2/10
2773/2773 [==============================] - 392s 141ms/sample - loss: 0.5442 - acc: 0.7559
Epoch 3/10
2773/2773 [==============================] - 392s 141ms/sample - loss: 0.5228 - acc: 0.7584
Epoch 4/10
2773/2773 [==============================] - 393s 142ms/sample - loss: 0.5045 - acc: 0.7613
Epoch 5/10
2773/2773 [==============================] - 388s 140ms/sample - loss: 0.4711 - acc: 0.7818
Epoch 6/10
2773/2773 [==============================] - 386s 139ms/sample - loss: 0.4244 - acc: 0.8092
Epoch 7/10
2773/2773 [==============================] - 392s 141ms/sample - loss: 0.3866 - acc: 0.8280
Epoch 8/10
2773/2773 [==============================] - 385s 139ms/sample - loss: 0.3290 - acc: 0.8612
Epoch 9/10
2773/2773 [==============================] - 383s 138ms/sample - loss: 0.2683 - acc: 0.8940
Epoch 10/10
2773/2773 [============================

In [ ]:
# decode test labels
y_test = decode(le_enc, y_test)
# decode predicted labels
y_preds = decode(le_enc, predicts)

In [ ]:
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77       495
           1       0.35      0.24      0.28       199

    accuracy                           0.65       694
   macro avg       0.54      0.53      0.53       694
weighted avg       0.62      0.65      0.63       694



In [ ]:
print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

Accuracy of ELMO is: 0.6527377521613833


Predicting for test data set

In [ ]:
df_test = pd.read_csv('taskA.En.input.csv',usecols=['text'],encoding='utf8')

In [ ]:
df = np.array(df_test['text'])

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./response-elmo-model.h5')
    #predicts = model.predict(x_test, batch_size=16)
    predictions = model.predict(df)

In [ ]:
predictions = predictions.round()

In [ ]:
result = []
for i in predictions:
  result.append(int(i[1]))

In [ ]:
np.savetxt("predictions.txt", result, fmt="%s")

In [ ]:
import pandas as pd
df_test = pd.read_csv('test.tsv', sep='\t', skiprows=1,header=None , names=['id','text','category'])
in_txt = open('predictions.txt', "r")
file_string = in_txt.read()
file_list = file_string.split(' ')retr

SyntaxError: ignored

In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': False,
    'max_seq_length': 75,
    'learning_rate': 0.00010,
    'weight_decay': 0.01,
    'warmup_ratio': 0.5,
    'max_grad_norm': 1.5,
    'num_train_epochs': 10,
    'train_batch_size': 40,
    'save_model_every_epoch': False,
    'save_steps': 4000,
    'fp16': True,
    'output_dir': '/outputs/',
    'evaluate_during_training': False,
}

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

In [ ]:
model = ClassificationModel('roberta', 'roberta-base', use_cuda=False, num_labels=2, args=train_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [ ]:
model.train_model(df_train)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/3107 [00:00<?, ?it/s]

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 1 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

...

Running Epoch 3 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 4 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/78 [00:00<?, ?it/s]

(546, 0.3919566530203822)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(df_eval, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/360 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/45 [00:00<?, ?it/s]

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(df_eval, acc=sklearn.metrics.f1_score)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/360 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/45 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
predictions, raw_outputs = model.predict(df_eval['tweet'].tolist())
print(predictions)
print(raw_outputs)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/360 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

[0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0
 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 1
 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0]
[[ 1.89177728 -1.31975269]
 [ 1.80845523 -1.2384491 ]
 [ 0.03779559  0.25428307]
 [ 1.64949954 -1.18136191]
 [ 1.96242189 -1.52101231]
 [ 2.11002445 -1.58843672]
 [ 1.94447494 -1.36527848]
 [ 2.03450441 -1.56376076]
 [ 2.08331108 -1.6186533 ]
 [ 1.36490929 -0.88081771]

In [ ]:
samples = ['I need my back and neck cracked differentlyyy',
           'Mustard has no business being zero calories',
           'I heard it might be snowing ??? Sounds fake, but okay??? Can anyone verify?????????'
          ]
predictions, _ = model.predict(samples)
label_dict = {0: 'not sarcastic', 1: 'sarcastic'}
for idx, sample in enumerate(samples):
  print('{}: {}, {}'.format(idx, sample, label_dict[predictions[idx]]))

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0: I need my back and neck cracked differentlyyy, not sarcastic
1: Mustard has no business being zero calories, sarcastic
2: I heard it might be snowing ??? Sounds fake, but okay??? Can anyone verify?????????, sarcastic
